# Verbatim opgavebeskrivelse
# Milestone 2
DESCRIPTION

After cleaning and processing our data in the first milestone, Milestone 2 will focus on how to efficiently represent the data in a database. Like last time, the milestone takes the form of a short jupyter notebook. It must be handed in on Friday, April 29, at 16:00 (in groups), and it is a requirement for attending the exam (it will be evaluated as passed/fail).


# Task 1.

The first task is to demonstrate that you have a working database containing the FakeNewsCorpus dataset. Explain your choice of schema design. You have been working on this task on a small subset of the data during the TA-sessions. For this milestone, demonstrate that your database contains a larger number of rows (e.g. one million - or however many you can reasonably work with on your available hardware), and that it supports simple queries.

In [59]:
# psh270, jxs535, fgp424, hkp680
import nltk as nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from nltk.corpus import wordnet
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('corpus')
nltk.download('all')
import numpy as np
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from cleantext.sklearn import CleanTransformer # likely required to ´pip install clean-text´
from cleantext import clean
from pathlib import Path
data = pd.read_csv("https://raw.githubusercontent.com/several27/FakeNewsCorpus/master/news_sample.csv")
#data = pd.read_csv(r"C:\Users\Computer\Documents\GitHub\Data-Science-Course-Work\CSVs\1mio-raw.csv")

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Computer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Computer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Error loading corpus: Package 'corpus' not found in index
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Computer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Computer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Computer\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]

In [60]:
def drop_useless_data(data):
    #Dropping unneeded columns
    cols_to_delete = ["Unnamed: 0"]
    for column in data.columns:
        if data[column].isnull().values.all():
            cols_to_delete.append(column)
    data.drop(cols_to_delete, 1, inplace=True)
    
    #Dropping entries with nan type
    data.dropna(subset = ["type"], inplace = True)
    #Dropping entries with unknown type
    data.drop(data.loc[data["type"] == "unknown"].index, inplace=True)

drop_useless_data(data)

C:\Users\Computer\AppData\Local\Temp\ipykernel_256\3917617433.py:7: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop(cols_to_delete, 1, inplace=True)


In [61]:
article_keyword_list = []
keyword_id = 0
article_author_list = [] #Ends up containing lists [article id, author name]
author_id = 0
tag_list = []
tags_id = 0
articles = []
has_written_list = []


def string_to_list(s):
    l = s.strip('][').split(', ')
    return [w.strip("'").lower() for w in l]

for i, article in data.iterrows():
    article_id = int(article["id"])

    if article["meta_keywords"] == "['']":
        article_keyword_list.append([keyword_id, article_id, np.nan])
        keyword_id += 1
    else:
        for word in string_to_list(article["meta_keywords"]):
            article_keyword_list.append([keyword_id, article_id, word])
            keyword_id += 1
    
    if str(article["authors"]) == "nan":
        article_author_list.append([author_id, article_id, np.nan])
        author_id += 1
    else:
        for author in string_to_list(article["authors"]):
            article_author_list.append([author_id, article_id, author])
            has_written_list.append([author_id, article_id])
            author_id += 1

    if str(article["tags"]) == "nan":
        tag_list.append([tags_id, article_id, np.nan])
        tags_id += 1
    else:
        for tag in string_to_list(article["tags"]):
            tag_list.append([tags_id, article_id, tag])
            tags_id += 1


    articles.append([article["id"],
                    article["domain"], 
                    article["type"],
                    article["url"],  
                    article["content"].replace("\n", ""),
                    article["scraped_at"],
                    article["inserted_at"],
                    article["updated_at"], 
                    article["title"],  
                    article["meta_description"]])

keywords_df = pd.DataFrame(article_keyword_list, columns=["id", "article_id", "keyword"])
authors_df = pd.DataFrame(article_author_list, columns=["id", "article_id", "author_name"])
tags_df = pd.DataFrame(tag_list, columns=["id", "article_id", "tag"])
article_df = pd.DataFrame(articles, columns=["article_id", "domain", "type", "url", "content", "scraped_at", "inserted_at", "updated_at", "title", "meta_description"])
has_written_df = pd.DataFrame(has_written_list, columns=["author_id", "article_id"])

keyword_combinations_path = Path(r"C:\Users\Computer\Documents\GitHub\Data-Science-Course-Work\CSVs\keywords.csv")
authors_path = Path(r"C:\Users\Computer\Documents\GitHub\Data-Science-Course-Work\CSVs\authors.csv")
tags_path = Path(r"C:\Users\Computer\Documents\GitHub\Data-Science-Course-Work\CSVs\tags.csv")
articles_path = Path(r"C:\Users\Computer\Documents\GitHub\Data-Science-Course-Work\CSVs\articles.csv")
has_written_path = Path(r"C:\Users\Computer\Documents\GitHub\Data-Science-Course-Work\CSVs\has_written.csv")

keywords_df.to_csv(keyword_combinations_path, index=False)
authors_df.to_csv(authors_path, index=False)
tags_df.to_csv(tags_path, index=False)
article_df.to_csv(articles_path, index=False)
has_written_df.to_csv(has_written_path, index=False)

# Task 2.

List the relations you have created in your database. For each relation:

list its attributes
list its functional dependencies.
list all the primary keys.
Is each relation in BCNF form? If not, show how to transform the tables in BCNF and explain why it might be better (or not) to use the BCNF relations in your database.





# Task 3.

Once your database is loaded, you can start issuing queries to better understand the characteristics of the data. Formulate the following queries in the database languages requested (in the square brackets following each item) and briefly discuss what you observe when you execute them over your database:   

List the domains of news articles of reliable type and scraped at or after January 15, 2018. NOTE: Do not include duplicate domains in your answer. [Languages: relational algebra and SQL]
List the name(s) of the most prolific author(s) of news articles of fake type. An author is among the most prolific if it has authored as many or more fake news articles as any other author in the dataset. [Languages: extended relational algebra and SQL]
Count the pairs of article IDs that exhibit the exact same set of meta-keywords, but only return the pairs where the set of meta-keywords is not empty. [Language: SQL]


# Task 4.

Now that we have our data in a database, let’s revisit the “interesting observations” task from Milestone 1 - but now using queries to the database. The idea is to write database queries (e.g. using GROUP BY and COUNT) that explore features of the data set that are relevant to the fake news prediction task: outliers, artefacts. It’s OK to investigate the same issues as in Milestone 1 (now using database queries) - but you are also very welcome to come up with completely new queries. You should write at least 3 such queries.

# Task 5.

Just like last time, after the hand in deadline, each group will be asked to evaluate the work of three other groups, based on a short list of criteria that you can find within the peergrade system. Again, this will only work well if everyone puts some effort into providing constructive comments, so please allocate some time to do this properly. It is an opportunity to get some feedback that can help you improve your final project. The deadline for giving feedback is a week after the hand-in deadline